# Import Dependencies

In [15]:
import pandas as pd
import os.path as osp
import os
from configs.general import \
                    MODELS, DATASETS, OPTIMS,\
                    INITIALIZATIONS, LR_SCHEDULERS, NPS, NSS,\
                    PHASES, EPOCHS, FOLDS, DEVICE,\
                    DATA_FILTERING_POLICIES, DATA_RETRIEVAL_POLICIES, \
                    EXPERIMENT_COLS, EXPERIMENT_BASE_DIR, EXPERIMENT_INFO_PATH,\
                    FILTERING_EXPERIMENT_INFO_PATH, FILTERING_EXPERIMENT_BASE_DIR, FILTERING_EXPERIMENT_COLS
                    

# Update experiments status

In [13]:
old_experiments = pd.read_csv(EXPERIMENT_INFO_PATH, index_col='index')
experiments = old_experiments.copy()
for index, row in experiments.iterrows():
    examin_dir = osp.join(EXPERIMENT_BASE_DIR, *[str(row[col]) for col in EXPERIMENT_COLS])
    if osp.isdir(examin_dir):
        number_of_folds = len(os.listdir(examin_dir))
        if number_of_folds == FOLDS:
            experiments.loc[index, 'done'] = True

print("total experiments:", len(old_experiments))
print("already done:", (old_experiments['done'] == True).sum())
print("required to update:", (experiments['done'] == True).sum() -(old_experiments['done'] == True).sum())


total experiments: 468
already done: 76
required to update: 0


In [ ]:
experiments.to_csv(EXPERIMENT_INFO_PATH)

# Update filtering experiments status

In [16]:
old_experiments = pd.read_csv(FILTERING_EXPERIMENT_INFO_PATH, index_col='index')
experiments = old_experiments.copy()
for index, row in experiments.iterrows():
    examin_dir = osp.join(FILTERING_EXPERIMENT_BASE_DIR, *[str(row[col]) for col in FILTERING_EXPERIMENT_COLS])
    if osp.isdir(examin_dir):
        number_of_folds = len(os.listdir(examin_dir))
        if number_of_folds == FOLDS:
            experiments.loc[index, 'done'] = True

print("total experiments:", len(old_experiments))
print("already done:", (old_experiments['done'] == True).sum())
print("required to update:", (experiments['done'] == True).sum() -(old_experiments['done'] == True).sum())


total experiments: 183
already done: 12
required to update: 0


In [ ]:
experiments.to_csv(FILTERING_EXPERIMENT_INFO_PATH)